In [1]:
import cirq
cirq.__version__

'0.7.0'

### Basics

In [2]:
# define the dimension of the grid
length = 3

In [3]:
# define qubits
qubits = [cirq.GridQubit(i,j) for i in range(length) for j in range(length)]
print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [4]:
# define circuit
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0)
circuit.append(cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1)
print(circuit)

(0, 0): ───H───

(0, 1): ───X───

(0, 2): ───H───

(1, 0): ───X───

(1, 1): ───H───

(1, 2): ───X───

(2, 0): ───H───

(2, 1): ───X───

(2, 2): ───H───


In [5]:
# alternative insert strategy
circuit = cirq.Circuit()
circuit.append((cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0), 
               strategy=cirq.InsertStrategy.EARLIEST)
circuit.append((cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1), 
               strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
for i, m in enumerate(circuit):
    print('Moment {}: {}'.format(i, m))

Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))


In [6]:
def rot_x_layer(length, half_turns):
    rot = cirq.XPowGate(exponent=half_turns)
    # rot = cos(half_turns * pi) I + i sin(half_turns * pi) X
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i,j))

In [7]:
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


### Setting up problem definition

In [8]:
import numpy as np

In [9]:
def rand2d(nrows, ncols):
    return np.random.choice([-1, 1], size=(nrows, ncols))

In [10]:
def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length, length-1)
    # links within a column
    jc = rand2d(length-1, length)
    return (h, jr, jc)

In [11]:
h, jr, jc = random_instance(3)

### Setting up quantum circuit

In [123]:
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield gate(cirq.GridQubit(i, j))

In [124]:
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr and jc fields."""
    gate = cirq.CZPowGate(exponent=half_turns)    
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j+1))
                
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i, j+1))
            
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j+1))

    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i+1, j))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i+1, j))
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i+1, j))

In [130]:
def create_ansatz(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    yield rot_x_layer(len(h), x_half_turns)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)

In [160]:
circuit = cirq.Circuit()
circuit.append(create_ansatz(h, jr, jc, 0.1, 0.2, 0.3))

### Run Simulator

In [161]:
simulator = cirq.Simulator()

In [162]:
%time result = simulator.simulate(circuit)

CPU times: user 30.5 ms, sys: 3.42 ms, total: 34 ms
Wall time: 20.7 ms


In [163]:
print(np.around(result.final_state, 3))

[ 0.883-0.14j  -0.022-0.14j  -0.126-0.064j -0.022+0.004j -0.126+0.064j
  0.01 +0.02j   0.01 +0.02j   0.003+0.002j -0.14 -0.022j -0.02 +0.01j
  0.016+0.016j  0.002+0.003j  0.022-0.004j  0.003-0.003j -0.001-0.004j
  0.   -0.001j -0.064-0.126j -0.02 +0.01j  -0.02 +0.01j  -0.002+0.003j
  0.016+0.016j  0.003-0.003j  0.004+0.001j  0.001-0.j    -0.016+0.016j
 -0.001+0.004j  0.003+0.003j  0.   +0.001j  0.002-0.003j -0.   -0.001j
 -0.   -0.001j  0.   -0.j    -0.126-0.064j -0.01 +0.02j   0.01 +0.02j
  0.003+0.002j  0.01 +0.02j   0.003-0.002j  0.003-0.002j  0.   -0.001j
  0.016+0.016j  0.004+0.001j -0.001-0.004j  0.   -0.001j -0.001-0.004j
 -0.   -0.j    -0.001+0.j    -0.   -0.j    -0.02 +0.01j   0.002+0.003j
  0.002+0.003j  0.001+0.j     0.004+0.001j  0.   -0.001j  0.001-0.j
  0.   -0.j     0.003+0.003j  0.001+0.j     0.   -0.j     0.   -0.j
 -0.   -0.001j -0.   -0.j    -0.   -0.j    -0.   -0.j    -0.14 -0.022j
 -0.004+0.022j  0.016+0.016j  0.004+0.001j  0.022-0.004j -0.001-0.004j
 -0.001-0.004j

In [157]:
circuit.append(cirq.measure(*qubits, key='x'))

In [158]:
%time result = simulator.run(circuit, repetitions=10000)

CPU times: user 184 ms, sys: 7.36 ms, total: 192 ms
Wall time: 160 ms


In [159]:
print(result.histogram(key='x'))

Counter({0: 7983, 256: 228, 8: 226, 64: 220, 2: 208, 4: 206, 128: 198, 1: 194, 16: 181, 32: 179, 17: 10, 288: 8, 65: 7, 48: 7, 160: 7, 80: 7, 24: 7, 36: 7, 258: 7, 192: 6, 40: 6, 12: 6, 3: 6, 384: 6, 272: 5, 20: 5, 129: 5, 132: 5, 264: 4, 6: 4, 72: 4, 257: 4, 136: 4, 68: 4, 96: 3, 260: 3, 66: 3, 34: 3, 10: 3, 320: 3, 18: 2, 130: 2, 33: 2, 261: 2, 5: 2, 52: 1, 193: 1, 14: 1, 274: 1, 336: 1, 22: 1, 9: 1, 266: 1})


In [19]:
def energy_func(h, jr, jc):
    def calc(measurements):
        zs = 1 - 2*measurements.reshape(*h.shape).astype(np.int8)
        tot_energy = np.sum(h * zs)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * zs[i, j] * zs[i, j+1]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * zs[i, j] * zs[i+1, j]
        return tot_energy
    return calc

In [33]:
print(result.histogram(key='x', fold_func=energy_func(h, jr, jc)))

Counter({1: 8816, -5: 447, -1: 259, 3: 225, 7: 214, -3: 10, -7: 8, 9: 6, -11: 6, -9: 5, 5: 4})


In [34]:
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions

In [35]:
print('Value of the objective function {}'.format(obj_func(results)))

Value of the objective function 0.8494


### Parameterize the Ansatz

In [112]:
import sympy
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')

In [113]:
circuit = cirq.Circuit()
circuit.append(create_ansatz(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))

In [114]:
resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.2, 'gamma': 0.3})
resolved_circuit = cirq.resolve_parameters(circuit, resolver)

In [106]:
obj_func(simulator.run(resolved_circuit, repetitions=10000))

0.8764

### Optimization

In [67]:
from scipy.optimize import minimize
import sys

In [107]:
def fom(ps):
    resolver = cirq.ParamResolver(dict(zip(['alpha', 'beta', 'gamma'], ps)))
    resolved_circuit = cirq.resolve_parameters(circuit, resolver)
    return obj_func(simulator.run(resolved_circuit, repetitions=100000))

In [108]:
def callback(ps):
    print(ps, fom(ps), file=sys.stderr)

In [109]:
%time fom([0.1, 0.2, 0.3])

CPU times: user 5.13 s, sys: 29.4 ms, total: 5.16 s
Wall time: 5.15 s


0.8592

In [88]:
res = minimize(
    fun      = fom,
    callback = callback,
    x0       = [0.1, 0.2, 0.3],
    method   = 'Nelder-Mead',
    options  = {'disp'   : 1,
                'maxiter': 10,
                'initial_simplex': [
                    [0.1, 0.2, 0.3],
                    [0.2, 0.2, 0.3],
                    [0.1, 0.3, 0.3],
                    [0.1, 0.2, 0.4],
                ],
                'xatol'  : 1e-2,
                'fatol'  : 3e-3
                }
)

[0.2 0.2 0.3] 0.5006
[0.26666667 0.06666667 0.46666667] 0.224
[0.43333333 0.13333333 0.33333333] -0.12028
[ 4.33333333e-01 -5.55111512e-17  3.66666667e-01] -0.13634
[ 4.33333333e-01 -5.55111512e-17  3.66666667e-01] -0.11788
[ 4.33333333e-01 -5.55111512e-17  3.66666667e-01] -0.0976
[ 4.33333333e-01 -5.55111512e-17  3.66666667e-01] -0.07834
[ 4.33333333e-01 -5.55111512e-17  3.66666667e-01] -0.14694


[ 4.33333333e-01 -5.55111512e-17  3.66666667e-01] -0.12318


In [93]:
print(res)

 final_simplex: (array([[ 4.33333333e-01, -5.55111512e-17,  3.66666667e-01],
       [ 4.04320988e-01,  4.90740741e-02,  3.98456790e-01],
       [ 4.48148148e-01,  2.77777778e-02,  4.01851852e-01],
       [ 4.33333333e-01,  1.33333333e-01,  3.33333333e-01]]), array([-0.15056, -0.14764, -0.1162 , -0.10598]))
           fun: -0.15056
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 20
           nit: 10
        status: 2
       success: False
             x: array([ 4.33333333e-01, -5.55111512e-17,  3.66666667e-01])


In [110]:
res1 = minimize(
    fun      = fom,
    callback = callback,
    x0       = [0.1, 0.2, 0.3],
    method   = 'L-BFGS-B',
    options  = {'disp'   : 1,
                'maxiter': 10,
                'ftol'   : 3e-3,
                'eps'    : 1e-2
                }
)

[0.43876062 0.10279798 0.18688693] -0.10656
[0.43876063 0.10279797 0.18688693] -0.15096
[0.43447729 0.09954065 0.1865424 ] -0.11308
[0.43447729 0.09954065 0.1865424 ] -0.13862


In [111]:
res1

      fun: -0.12452
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([1.042, 2.616, 1.886])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 108
      nit: 4
   status: 0
  success: True
        x: array([0.43447729, 0.09954065, 0.1865424 ])

### Interfacing with QASM format

In [91]:
from cirq.contrib.qasm_import import circuit_from_qasm
circuit = circuit_from_qasm("""
    OPENQASM 2.0;
    include "qelib1.inc";
    qreg q[3];
    creg meas[3];
    h q;
    measure q -> meas;
    """)
print(circuit)

q_0: ───H───M('meas_0')───

q_1: ───H───M('meas_1')───

q_2: ───H───M('meas_2')───


### Monte Carlo simulations of noise

In [170]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.bit_flip(p=0.2)(q), cirq.measure(q))
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)
print(result.histogram(key='a'))

Counter({0: 803, 1: 197})


In [173]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit, repetitions=1000)
print(result.histogram(key='a'))

Counter({0: 597, 1: 403})


### Device

In [174]:
import cirq
from cirq.devices import GridQubit

In [175]:
class Xmon10Device(cirq.Device):
    def __init__(self):
        self.qubits = [GridQubit(i, 0) for i in range(10)]

    def validate_operation(self, operation):
        if not isinstance(operation, cirq.GateOperation):
            raise ValueError('{!r} is not a supported operation'.format(operation))
        if not isinstance(operation.gate, (cirq.CZPowGate,
                                           cirq.XPowGate,
                                           cirq.PhasedXPowGate,
                                           cirq.YPowGate)):
            raise ValueError('{!r} is not a supported gate'.format(operation.gate))
        if len(operation.qubits) == 2:
            p, q = operation.qubits
            if not p.is_adjacent(q):
                raise ValueError('Non-local interaction: {}'.format(repr(operation)))

    def validate_circuit(self, circuit):
        for moment in circuit:
            for operation in moment.operations:
                self.validate_operation(operation)

In [177]:
device = Xmon10Device()
circuit = cirq.Circuit()
circuit.append([cirq.CZ(device.qubits[0], device.qubits[2])])
try: 
    device.validate_circuit(circuit)
except ValueError as e:
    print(e)

Non-local interaction: cirq.CZ.on(cirq.GridQubit(0, 0), cirq.GridQubit(2, 0))
